# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import psycopg2
import pandas as pd
import configparser
from time import time
import boto
import s3fs
import os
from load_fact_and_dim_tables import insert_dim_queries, insert_fact_queries, create_table_queries, copy_table_queries, truncate_tables, drop_tables

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>
Scope of the project is to create a datawarehouse where the subject matter is crime staticitis in 2 major cites: New York City and Los Angeles. Based on the data gather, we can make comparsion based on the historical data provided.  
The data are in csv format and loaded to s3 bucket where the data will be loaded to Redshift

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
The data come from data.gov which has historical data on NYC crime and LA crime.
The data consist of record date of the report crime, suspect and victim gender, race, location and type of crime

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [3]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()  

In [4]:
ARN = config.get("IAM_ROLE","ARN")
la_data = config.get("S3","LA_DATA_FILE")
nyc_data = config.get("S3","NYC_DATA_FILE")

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [5]:
dfnyc = pd.read_csv('s3://andre.lee.de/NYPD_Complaint_Data_Historic.csv', nrows=1000)

In [6]:
dfla = pd.read_csv('s3://andre.lee.de/LAX_Crime_Data_from_2010_to_Present.csv', nrows=1000)

In [7]:
dfnyc.head(10)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,491097831,08/31/2013,20:00:00,09/02/2013,13:10:00,76,09/03/2013,578,HARRASSMENT 2,638.0,...,F,NaN,40.684084,-73.986780,"(40.684084156, -73.98677974)",PATROL BORO BKLYN SOUTH,NaN,NaN,UNKNOWN,F
1,827796420,08/31/2013,19:45:00,08/31/2013,20:00:00,40,09/03/2013,359,OFFENSES AGAINST PUBLIC ADMINI,759.0,...,NaN,NaN,40.815606,-73.914579,"(40.815606285, -73.914578735)",PATROL BORO BRONX,NaN,NaN,UNKNOWN,E
2,823404713,08/31/2013,19:30:00,08/31/2013,20:00:00,10,09/03/2013,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,...,U,NaN,40.747810,-73.998518,"(40.747810021, -73.998518132)",PATROL BORO MAN SOUTH,NaN,18-24,WHITE HISPANIC,F
3,950495742,08/31/2013,19:00:00,09/02/2013,09:00:00,106,09/03/2013,110,GRAND LARCENY OF MOTOR VEHICLE,441.0,...,NaN,NaN,40.662512,-73.856311,"(40.662511814, -73.85631094)",PATROL BORO QUEENS SOUTH,NaN,45-64,WHITE HISPANIC,F
4,655454255,08/31/2013,19:00:00,09/03/2013,11:00:00,71,09/03/2013,107,BURGLARY,234.0,...,NaN,NaN,40.666588,-73.939678,"(40.666587627, -73.939677686)",PATROL BORO BKLYN SOUTH,NaN,25-44,BLACK,M
5,777845621,08/31/2013,18:00:00,09/03/2013,14:30:00,81,09/03/2013,341,PETIT LARCENY,338.0,...,NaN,NaN,40.686719,-73.923470,"(40.686719153, -73.923469723)",PATROL BORO BKLYN NORTH,NaN,25-44,BLACK,M
6,766449786,08/31/2013,17:00:00,08/31/2013,17:10:00,120,09/03/2013,578,HARRASSMENT 2,637.0,...,M,NaN,40.646466,-74.089298,"(40.646465992, -74.089297652)",PATROL BORO STATEN ISLAND,NaN,25-44,WHITE HISPANIC,F
7,710053577,08/31/2013,17:00:00,09/03/2013,08:30:00,108,09/03/2013,351,CRIMINAL MISCHIEF & RELATED OF,258.0,...,U,NaN,40.746308,-73.915696,"(40.746307995, -73.915696141)",PATROL BORO QUEENS NORTH,NaN,65+,WHITE,M
8,980715415,08/31/2013,15:59:00,09/02/2013,16:37:00,13,09/03/2013,107,BURGLARY,214.0,...,NaN,NaN,40.738834,-73.989861,"(40.738834251, -73.98986141)",PATROL BORO MAN SOUTH,NaN,NaN,UNKNOWN,D
9,797919473,08/31/2013,15:49:00,08/31/2013,20:47:00,120,09/03/2013,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,...,F,NaN,40.633107,-74.086448,"(40.633106542, -74.086447943)",PATROL BORO STATEN ISLAND,NaN,25-44,BLACK HISPANIC,F


In [8]:
dfla.head(10)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488
5,100100506,01/05/2010 12:00:00 AM,01/04/2010 12:00:00 AM,1650,1,Central,162,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),...,AA,Adult Arrest,442,NaN,NaN,NaN,700 W 7TH ST,NaN,34.0480,-118.2577
6,100100508,01/08/2010 12:00:00 AM,01/07/2010 12:00:00 AM,2005,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330,NaN,NaN,NaN,PICO BL,GRAND AV,34.0389,-118.2643
7,100100509,01/09/2010 12:00:00 AM,01/08/2010 12:00:00 AM,2100,1,Central,157,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,AA,Adult Arrest,230,NaN,NaN,NaN,500 CROCKER ST,NaN,34.0435,-118.2427
8,100100510,01/09/2010 12:00:00 AM,01/09/2010 12:00:00 AM,230,1,Central,171,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230,NaN,NaN,NaN,800 W OLYMPIC BL,NaN,34.0450,-118.2640
9,100100511,01/09/2010 12:00:00 AM,01/06/2010 12:00:00 AM,2100,1,Central,132,1,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",...,IC,Invest Cont,341,998.0,NaN,NaN,200 S OLIVE ST,NaN,34.0538,-118.2488


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Covert date from varchar to date
seperate out the key data into different dimenison tables

In [9]:
dfnyc2 = pd.to_datetime(dfnyc['CMPLNT_FR_DT'])

In [10]:
dfnyc2.head(1)

0   2013-08-31
Name: CMPLNT_FR_DT, dtype: datetime64[ns]

In [11]:
dfnyc['CMPLNT_FR_DT'].head(1)

0    08/31/2013
Name: CMPLNT_FR_DT, dtype: object

In [12]:
dfla2 = pd.to_datetime(dfla['Date Rptd'])

In [13]:
dfla2.head(1)

0   2010-02-20
Name: Date Rptd, dtype: datetime64[ns]

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The conceptual data model is based on star data model design.  The star data model design is simplic and easy to join fact and dimenional tables based on the relationship.

#### 3.2 Mapping Out Data Pipelines
-
    1.  Copy data from the s3 bucket to staging tables in Redshift
    2.  Extract the necessary data from the staging tables and load it to the dimension tables
    3.  Extract the necessary data from the staging table and load to the fact table
    

In [14]:
#decide weather to truncate the data or drop the tables or do nothing
action = 2

if action == 1:
    for query in truncate_tables:
        print(query)
        cur.execute(query)
    print('DONE!!!')        
elif action == 2:
    for query in drop_tables:
        print(query)
        cur.execute(query)
    print('DONE!!!')
elif action == 3:
    print ('No Action Taken')

DROP TABLE IF EXISTS D_AGE_GROUP;
DROP TABLE IF EXISTS D_AREA;
DROP TABLE IF EXISTS D_COORDINATES;
DROP TABLE IF EXISTS D_GENDER_TYPE;
DROP TABLE IF EXISTS D_JURISDICTION;
DROP TABLE IF EXISTS D_PDCODES;
DROP TABLE IF EXISTS D_RACE_TYPE;
DROP TABLE IF EXISTS D_US_CITIES;
DROP TABLE IF EXISTS F_CRIME_DATA;
DROP TABLE IF EXISTS STAGING_LA_CRIME;
DROP TABLE IF EXISTS STAGING_NYC_CRIME;
DROP TABLE IF EXISTS STAGING_US_CITIES;
DONE!!!


### Create tables

In [15]:
if action == 2:
    print('Recreate tables since the tables were dropped')
    for query in create_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()
    
    print('DONE!!!')    

Recreate tables since the tables were dropped
CREATE TABLE IF NOT EXISTS d_age_group
(
    id integer identity(0, 1) PRIMARY KEY unique not null,
    age_group varchar(10) ,
    age integer
);
CREATE TABLE IF NOT EXISTS d_area
(
    id integer identity(0, 1) PRIMARY KEY unique not null,
    area_cd char(3),
    area_name varchar(50) 
);
CREATE TABLE IF NOT EXISTS d_coordinates
(
    id integer identity(0, 1) PRIMARY KEY unique not null,
    latitude varchar,
    longitude varchar,
    lat_lon varchar(60) 
);
CREATE TABLE IF NOT EXISTS d_gender_type
(
    id integer identity(0, 1) PRIMARY KEY unique not null,
    gender character(1) ,
    gender_desc varchar(20) 
);
CREATE TABLE IF NOT EXISTS d_jurisdiction
(
    id integer identity(0, 1) PRIMARY KEY unique not null,
    juris_cd integer,
    juris_desc varchar(100) 
);
CREATE TABLE IF NOT EXISTS d_race_type
(
    id integer identity(0, 1) PRIMARY KEY unique not null,
    race varchar(50) 
);
CREATE TABLE IF NOT EXISTS d_us_cities
(
   

### Load staging tables
    

In [16]:
for query in copy_table_queries:
    loadTimes = []

    print('======= LOAD Staging tables =======')
    print(query)  
    t0 = time()          
    cur.execute(query)         
    conn.commit()
    loadTime = time()-t0          
    loadTimes.append(loadTime)
    print("=== DONE IN: {0:.2f} sec\n".format(loadTime))  

======= LOAD Staging tables =======
copy staging_la_crime from 's3://andre.lee.de/LAX_Crime_Data_from_2010_to_Present.csv'
    iam_role 'arn:aws:iam::915851018597:role/myRedshiftRole'
    CSV IGNOREHEADER as 1 region 'us-east-2';

=== DONE IN: 32.29 sec

======= LOAD Staging tables =======
copy staging_nyc_crime from 's3://andre.lee.de/NYPD_Complaint_Data_Historic.csv'
    iam_role 'arn:aws:iam::915851018597:role/myRedshiftRole'
    CSV IGNOREHEADER as 1 region 'us-east-2';

=== DONE IN: 98.13 sec

======= LOAD Staging tables =======
copy staging_us_cities from 's3://andre.lee.de/us-zip-code-latitude-and-longitude.csv'
    iam_role 'arn:aws:iam::915851018597:role/myRedshiftRole'
    DELIMITER ';' IGNOREHEADER as 1 region 'us-east-2';

=== DONE IN: 1.07 sec



In [17]:
conn.commit()

### Load dimension tables
    

In [18]:
#from load_fact_and_dim_tables import insert_dim_queries, insert_fact_queries

In [19]:
for query in insert_dim_queries:
    loadTimes = []

    print('======= LOAD Dimension tables =======')
    print(query)  
    t0 = time()          
    cur.execute(query)         
    conn.commit()
    loadTime = time()-t0          
    loadTimes.append(loadTime)
    print("=== DONE IN: {0:.2f} sec\n".format(loadTime))   

======= LOAD Dimension tables =======
INSERT INTO D_AREA
(AREA_CD, AREA_NAME) 
SELECT distinct 'NYC' AS AREA_CD, boro_nm FROM STAGING_NYC_CRIME
where boro_nm is not null
union
select distinct 'LAX' AS AREA_CD, area_name from staging_la_crime;
=== DONE IN: 0.36 sec

======= LOAD Dimension tables =======
insert into d_coordinates
(latitude, longitude, lat_lon)
select distinct
	latitude,
	longitude,
	lat_lon
from	
staging_nyc_crime;
=== DONE IN: 0.83 sec

======= LOAD Dimension tables =======
insert into d_race_type
(race)
select distinct 
    case 
        when susp_race = ' ' then 'N/A'
        else susp_race end as susp_race 
from staging_nyc_crime where susp_race is not null
=== DONE IN: 0.30 sec

======= LOAD Dimension tables =======
insert into public.d_jurisdiction
(juris_cd, juris_desc)
select distinct jurisdiction_code, juris_desc from staging_nyc_crime
order by 1;
=== DONE IN: 0.32 sec

======= LOAD Dimension tables =======
insert into d_gender_type
(gender,gender_desc)
values('

In [20]:
insert_dim_queries

["INSERT INTO D_AREA\n(AREA_CD, AREA_NAME) \nSELECT distinct 'NYC' AS AREA_CD, boro_nm FROM STAGING_NYC_CRIME\nwhere boro_nm is not null\nunion\nselect distinct 'LAX' AS AREA_CD, area_name from staging_la_crime;",
 'insert into d_coordinates\n(latitude, longitude, lat_lon)\nselect distinct\n\tlatitude,\n\tlongitude,\n\tlat_lon\nfrom\t\nstaging_nyc_crime;',
 "insert into d_race_type\n(race)\nselect distinct \n    case \n        when susp_race = ' ' then 'N/A'\n        else susp_race end as susp_race \nfrom staging_nyc_crime where susp_race is not null",
 'insert into public.d_jurisdiction\n(juris_cd, juris_desc)\nselect distinct jurisdiction_code, juris_desc from staging_nyc_crime\norder by 1;',
 "insert into d_gender_type\n(gender,gender_desc)\nvalues('M','Male');",
 "insert into d_gender_type\n(gender,gender_desc)\nvalues('F','Female');",
 "insert into d_gender_type\n(gender,gender_desc)\nvalues('U','UNKNOWN');",
 "insert into d_age_group\n(AGE_GROUP, AGE)\nselect distinct \n'UNKNOWN'

In [21]:
conn.commit()

### Load fact table    

In [22]:
for query in insert_fact_queries:
    loadTimes = []

    print('======= LOAD fact tables =======')
    print(query)  
    t0 = time()          
    cur.execute(query)         
    conn.commit()
    loadTime = time()-t0          
    loadTimes.append(loadTime)
    print("=== DONE IN: {0:.2f} sec\n".format(loadTime))   

======= LOAD fact tables =======
insert into f_crime_data
(
crime_location,rpt_date_of_crime,crime_code,crime_desc,
	suspect_race_id,suspect_age_id,vic_age_id,vic_race_id,
	area_or_boro,premis_desc,location_id
)
select distinct
'LAX' AS crime_location,
to_date(date_occ,'MM/DD/YYYY') as rpt_date_of_crime,
cast(crm_cd as integer) as crime_code,
crm_cd_desc as crime_desc,
6 as suspect_race_id,
1 as suspect_age_id,
case
	when stg.vict_age is null then 1
	else age_grp.id
end as vic_age_id,
CASE
	WHEN vic_rt.race is null then '6'
	else vic_rt.id
END as vic_race_id,
area_name as area_or_boro,
premis_desc,
case
	when loc.id is null then -1
	else loc.id
end as location_id
from 
	staging_la_crime stg
	LEFT join (select distinct id, substring(race,1,1) race from d_race_type) vic_rt
		on vic_rt.race = stg.vict_descent
	LEFT join d_age_group age_grp
		on age_grp.age = cast(stg.vict_age as integer)
	LEFT join d_coordinates loc
		on loc.latitude = stg.lat 
		and loc.longitude = stg.lon
order by 2
===

In [23]:
conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [24]:
sql_count = """
select count(*), crime_location from f_crime_data
group by crime_location;
"""

In [25]:
sql_dup_chk = """
select * from (
select latitude, longitude, count(*) over(partition by latitude, longitude) cnt from  d_coordinates )x
where cnt >1
"""

In [26]:
dfCount = pd.read_sql(sql_count, conn)
dfCount

,count,crime_location
0,6394277,NYC
1,3104606,LAX


In [27]:
dfDup = pd.read_sql(sql_dup_chk, conn)

In [28]:
dfDup

,latitude,longitude,cnt


In [29]:
dfStgNyc = pd.read_sql('select * from staging_nyc_crime limit 10', conn)

In [30]:
dfStgNyc

,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,...,susp_sex,transit_district,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex
0,950495742.0,08/31/2013,19:00:00,09/02/2013,09:00:00,106,09/03/2013,110.0,GRAND LARCENY OF MOTOR VEHICLE,441,...,,None,40.662511814,-73.85631094,"(40.662511814, -73.85631094)",PATROL BORO QUEENS SOUTH,,45-64,WHITE HISPANIC,F
1,301678035.0,08/31/2013,15:30:00,09/03/2013,09:45:00,73,09/03/2013,110.0,GRAND LARCENY OF MOTOR VEHICLE,455,...,M,None,40.656002929,-73.909178756,"(40.656002929, -73.909178756)",PATROL BORO BKLYN NORTH,,,UNKNOWN,D
2,546986419.0,08/31/2013,13:00:00,09/03/2013,13:00:00,25,09/03/2013,107.0,BURGLARY,221,...,,None,40.809858783,-73.935624768,"(40.809858783, -73.935624768)",PATROL BORO MAN NORTH,,25-44,BLACK,F
3,227010732.0,08/31/2013,10:00:00,09/03/2013,16:30:00,28,09/03/2013,341.0,PETIT LARCENY,338,...,F,None,40.798146087,-73.950357672,"(40.798146087, -73.950357672)",PATROL BORO MAN NORTH,,<18,BLACK,M
4,869211304.0,08/31/2013,04:00:00,08/31/2013,04:30:00,67,09/03/2013,233.0,SEX CRIMES,175,...,M,None,40.648650085,-73.950335563,"(40.648650085, -73.950335563)",PATROL BORO BKLYN SOUTH,,18-24,BLACK,F
5,488442382.0,08/30/2013,23:00:00,08/30/2013,23:05:00,68,09/03/2013,578.0,HARRASSMENT 2,638,...,M,None,40.616599871,-74.03233972,"(40.616599871, -74.03233972)",PATROL BORO BKLYN SOUTH,,25-44,BLACK,F
6,404963463.0,08/30/2013,20:00:00,08/30/2013,20:05:00,25,09/03/2013,341.0,PETIT LARCENY,339,...,,None,40.787874599,-73.928181646,"(40.787874599, -73.928181646)",PATROL BORO MAN NORTH,,18-24,WHITE,F
7,648426615.0,08/30/2013,18:35:00,,,63,09/03/2013,341.0,PETIT LARCENY,338,...,,None,40.615283892,-73.916293835,"(40.615283892, -73.916293835)",PATROL BORO BKLYN SOUTH,,65+,WHITE HISPANIC,F
8,372845558.0,08/30/2013,17:00:00,09/03/2013,09:00:00,14,09/03/2013,107.0,BURGLARY,214,...,,None,40.752612565,-73.986628988,"(40.752612565, -73.986628988)",PATROL BORO MAN SOUTH,,,UNKNOWN,D
9,486310531.0,08/30/2013,16:00:00,09/03/2013,08:00:00,114,09/03/2013,109.0,GRAND LARCENY,438,...,,None,40.754933481,-73.910390193,"(40.754933481, -73.910390193)",PATROL BORO QUEENS NORTH,,,UNKNOWN,D


In [31]:
dfloc = pd.read_sql('select * from staging_us_cities limit 19', conn)

In [32]:
dfloc

,zip,city,state,latitude,longitude,timezone,dst,geopoint
0,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"
1,41065,Muses Mills,KY,38.3481,-83.718626,-5,1,"38.3481,-83.718626"
2,20227,Washington,DC,38.893311,-77.014647,-5,1,"38.893311,-77.014647"
3,26253,Beverly,WV,38.827205,-79.89057,-5,1,"38.827205,-79.89057"
4,62090,Venice,IL,38.67065,-90.17052,-6,1,"38.67065,-90.17052"
5,63179,Saint Louis,MO,38.6531,-90.243462,-6,1,"38.6531,-90.243462"
6,81148,Romeo,CO,37.171583,-105.98648,-7,1,"37.171583,-105.98648"
7,93234,Huron,CA,36.209815,-120.0847,-8,1,"36.209815,-120.0847"
8,43984,New Rumley,OH,40.29649,-81.102502,-5,1,"40.29649,-81.102502"
9,31732,Cedar Springs,GA,31.169794,-85.0557,-5,1,"31.169794,-85.0557"


In [33]:
#conn.close()

In [34]:
query_chk_load_cnt = """
select count(*) as total, 'd_age_group' as table_nm from d_age_group
union
select count(*) as total, 'd_area' as table_nm from d_area
union
select count(*) as total, 'd_coordinates' as table_nm from d_coordinates
union
select count(*) as total, 'd_gender_type' as table_nm from d_gender_type
union
select count(*) as total, 'd_jurisdiction' as table_nm from d_jurisdiction
union
select count(*) as total, 'd_race_type' as table_nm from d_race_type
union
select count(*) as total, 'd_us_cities' as table_nm from d_us_cities
union
select count(*) as total, 'f_crime_data' as table_nm from f_crime_data;
"""

In [35]:
dfCnt = pd.read_sql(query_chk_load_cnt, conn)

In [36]:
dfCnt

,total,table_nm
0,184302,d_coordinates
1,27,d_area
2,111,d_age_group
3,3,d_gender_type
4,43191,d_us_cities
5,9498883,f_crime_data
6,29,d_jurisdiction
7,9,d_race_type


In [37]:
#pd.read_sql('select * from d_pdcodes;', conn).head(10)

In [38]:
pd.read_sql('select * from d_race_type',conn)

,id,race
0,2,BLACK
1,12,WHITE
2,10,ASIAN / PACIFIC ISLANDER
3,14,OTHER
4,7,BLACK HISPANIC
5,0,WHITE HISPANIC
6,4,N/A
7,6,UNKNOWN
8,20,AMERICAN INDIAN/ALASKAN NATIVE


In [39]:
sql_crime_by_race_and_year = """
select count(*) total, 
    suspect_race_id,
	  race as suspect_race, 
	  to_char(rpt_date_of_crime,'YYYY') as Year 
from f_crime_data f
	left join  d_race_type d
		on d.id = suspect_race_id
where crime_location = 'NYC'
	  and rpt_date_of_crime between '2012-01-01' and '2012-12-31'
group by suspect_race_id , race,  to_char(rpt_date_of_crime,'YYYY')
"""

In [40]:
pd.read_sql(sql_crime_by_race_and_year, conn)

,total,suspect_race_id,suspect_race,year
0,285923,4,N/A,2012
1,32716,0,WHITE HISPANIC,2012
2,77197,2,BLACK,2012
3,60012,6,UNKNOWN,2012
4,10171,7,BLACK HISPANIC,2012
5,23070,12,WHITE,2012
6,5563,10,ASIAN / PACIFIC ISLANDER,2012
7,656,20,AMERICAN INDIAN/ALASKAN NATIVE,2012


In [41]:
pd.read_sql("""
select distinct 
    case 
        when susp_race = ' ' then 'N/A'
        else susp_race end as susp_race 
from staging_nyc_crime where susp_race is not null
""",conn)

,susp_race
0,WHITE HISPANIC
1,BLACK
2,ASIAN / PACIFIC ISLANDER
3,UNKNOWN
4,OTHER
5,N/A
6,WHITE
7,AMERICAN INDIAN/ALASKAN NATIVE
8,BLACK HISPANIC


In [42]:
pd.read_sql('select * from d_race_type order by 2', conn)

,id,race
0,20,AMERICAN INDIAN/ALASKAN NATIVE
1,10,ASIAN / PACIFIC ISLANDER
2,2,BLACK
3,7,BLACK HISPANIC
4,4,N/A
5,14,OTHER
6,6,UNKNOWN
7,12,WHITE
8,0,WHITE HISPANIC


In [43]:
sql_crime_by_race_and_year = """
select count(*) as total, 
	   crime_location, 
	   case 
		when area_or_boro is null then 'N/A'
		when area_or_boro = ' ' then 'N/A'
		else area_or_boro
	   end as area_or_boro,
	   to_char(rpt_date_of_crime,'YYYY') as Year
from f_crime_data f
WHERE rpt_date_of_crime BETWEEN '2014-01-01' and '2016-12-31'
group by crime_location, area_or_boro, to_char(rpt_date_of_crime,'YYYY')
order by 4,1
"""

In [44]:
pd.read_sql_query(sql_crime_by_race_and_year,conn).head(10)

,total,crime_location,area_or_boro,year
0,326,NYC,N/A,2014
1,8674,LAX,Hollenbeck,2014
2,9532,LAX,Foothill,2014
3,11677,LAX,Rampart,2014
4,12044,LAX,Harbor,2014
5,12676,LAX,Newton,2014
6,13029,LAX,Mission,2014
7,13058,LAX,West Valley,2014
8,13077,LAX,Olympic,2014
9,13289,LAX,Topanga,2014


In [45]:
conn.commit()